In [66]:
import time
import datetime as dt

from pykrx import stock
import pandas as pd

In [67]:
dt_now = str(dt.datetime.now().date())
print(f'{dt_now} 기준')
dt_now = ''.join(c for c in dt_now if c not in '-')

df = stock.get_market_fundamental_by_ticker(date=dt_now)
df.head()

2023-10-10 기준


,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
095570,8076,19.703125,0.489990,201,6.820312,270
006840,41948,0.000000,0.399902,0,1.190430,200
027410,16528,13.640625,0.199951,247,3.259766,110
282330,55724,12.460938,2.509766,11203,2.939453,4100
138930,30468,2.939453,0.229980,2404,8.851562,625


In [68]:
df = df[['PER', 'PBR']]
df.head(5)

,PER,PBR
티커,,
095570,19.703125,0.489990
006840,0.000000,0.399902
027410,13.640625,0.199951
282330,12.460938,2.509766
138930,2.939453,0.229980


In [69]:
df.isnull().sum()

PER    0
PBR    0
dtype: int64

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 922 entries, 095570 to 003280
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PER     922 non-null    float16
 1   PBR     922 non-null    float16
dtypes: float16(2)
memory usage: 10.8+ KB


In [71]:
del_index = df[(df['PBR'] <= 0.2) | (df['PER'] <= 0)].index
df = df.drop(del_index)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 583 entries, 095570 to 003280
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PER     583 non-null    float16
 1   PBR     583 non-null    float16
dtypes: float16(2)
memory usage: 6.8+ KB


In [73]:
df.describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1])

,PER,PBR
count,583.000000,583.000000
mean,36.906250,1.056641
std,inf,1.532227
min,0.649902,0.209961
10%,2.982422,0.290039
20%,4.407813,0.350098
30%,6.198437,0.429932
40%,8.187500,0.527832
50%,9.929688,0.620117
60%,12.076562,0.779785


In [74]:
def change_to_rank(df, col, val):
    if val <= df[col].quantile(.1):
        return 10
    elif val <= df[col].quantile(.2):
        return 9
    elif val <= df[col].quantile(.3):
        return 8
    elif val <= df[col].quantile(.4):
        return 7
    elif val <= df[col].quantile(.5):
        return 6
    elif val <= df[col].quantile(.6):
        return 5
    elif val <= df[col].quantile(.7):
        return 4
    elif val <= df[col].quantile(.8):
        return 3
    elif val <= df[col].quantile(.9):
        return 2
    else :
        return 1

In [75]:
rank_df = pd.DataFrame(columns=['PER', 'PBR'])

for col in df.columns:
    rank_df[col] = df[col].apply(lambda x : change_to_rank(df, col, x))
rank_df

,PER,PBR
티커,,
095570,3,7
282330,4,1
138930,10,10
001460,7,8
001040,4,8
...,...,...
081660,7,3
005870,6,6
005010,10,9


In [76]:
rank_df['rank_sum'] = rank_df.sum(axis=1)
rank_df = rank_df.sort_values('rank_sum', ascending=False)
rank_df


,PER,PBR,rank_sum
티커,,,
093050,10,10,20
375500,10,10,20
001230,10,10,20
029460,10,10,20
000670,10,10,20
...,...,...,...
006110,1,1,2
352820,1,1,2
003720,1,1,2


In [77]:
num = 50
selected_stock_df = df.loc[rank_df.iloc[:50].index]
selected_stock_df

,PER,PBR
티커,,
093050,2.580078,0.270020
375500,2.849609,0.250000
001230,2.089844,0.270020
029460,1.990234,0.280029
000670,2.400391,0.229980
006660,1.750000,0.280029
017940,2.330078,0.229980
377740,1.330078,0.280029
007860,2.289062,0.280029


In [78]:
selected_ticker_list = selected_stock_df.index
selected_stocks = {}
for ticker in selected_ticker_list:
    stock_name = stock.get_market_ticker_name(ticker)
    selected_stocks[ticker] = stock_name
    print(stock_name)


LF
DL이앤씨
동국홀딩스
케이씨
영풍
삼성공조
E1
바이오노트
서연
흥국화재
인터지스
코오롱
일신방직
BNK금융지주
SK디스커버리
대한제강
현대차증권
동부건설
HMM
신영와코루
한섬
SIMPAC
티웨이홀딩스
KISCO홀딩스
한국전자홀딩스
한일홀딩스
에스디바이오센서
SNT홀딩스
DSR
GS건설
한화
DGB금융지주
유성기업
다우기술
휴스틸
DSR제강
우리금융지주
GS
LX홀딩스
LX인터내셔널
광주신세계
무림페이퍼
사조대림
현대홈쇼핑
한화생명
사조산업
미래에셋증권
금강공업
문배철강
현대제철


In [86]:
selected_stock_df.to_csv('selected_df.csv', index=True)

In [87]:
test_df = pd.read_csv('selected_df.csv', index_col=0)

In [88]:
test_df.head()

,PER,PBR
티커,,
93050,2.58,0.27
375500,2.85,0.25
1230,2.09,0.27
29460,1.99,0.28
670,2.40,0.23


In [91]:
test_list = test_df.index
test_stocks = {}
print(test_list)
for ticker in test_list:
    print(str(ticker).zfill(6))
    stock_name = stock.get_market_ticker_name(str(ticker).zfill(6))
    print(stock_name)
    test_stocks[ticker] = stock_name
    # print(stock_name)

Int64Index([ 93050, 375500,   1230,  29460,    670,   6660,  17940, 377740,
              7860,    540, 129260,   2020,   3200, 138930,   6120,  84010,
              1500,   5960,  11200,   5800,  20000,   9160,   4870,   1940,
              6200,   3300, 137310,  36530, 155660,   6360,    880, 139130,
              2920,  23590,   5010,  69730, 316140,  78930, 383800,   1120,
             37710,   9200,   3960,  57050,  88350,   7160,   6800,  14280,
              8420,   4020],
           dtype='int64', name='티커')
093050
LF
375500
DL이앤씨
001230
동국홀딩스
029460
케이씨
000670
영풍
006660
삼성공조
017940
E1
377740
바이오노트
007860
서연
000540
흥국화재
129260
인터지스
002020
코오롱
003200
일신방직
138930
BNK금융지주
006120
SK디스커버리
084010
대한제강
001500
현대차증권
005960
동부건설
011200
HMM
005800
신영와코루
020000
한섬
009160
SIMPAC
004870
티웨이홀딩스
001940
KISCO홀딩스
006200
한국전자홀딩스
003300
한일홀딩스
137310
에스디바이오센서
036530
SNT홀딩스
155660
DSR
006360
GS건설
000880
한화
139130
DGB금융지주
002920
유성기업
023590
다우기술
005010
휴스틸
069730
DSR제강
316140
우리금융지주
078930
GS
383800